In [ ]:
import numpy as np
from numba import jit, vectorize, cuda, guvectorize, float32,int32
import numba
import math 
import random

In [ ]:
def initial_pop_generation(x, n, start_wth_zeros=False, min_value_n=None,
                           max_value_n=None):
    '''
    x : int
        parzysta liczba osobników
    n : int
        parzysta liczba genów
    start_wth_zeros : bool
        True - początkowe cechy osobnika równe zero
        False - losowo dobrane cechy początkowe
    min_value_n : int
        dolne ogarniczenie wartości genu
    max_value_n : int
        górne ograniczenie wartości genu
    '''
    population = []
    if start_wth_zeros:
        for i in range(x):
            individual = np.array([0 for i in range(0, n)])
            population.append(individual)
    else:
        if (min_value_n == None or max_value_n == None):
            raise Exception('Set min&max value for n or set start_with_zeros to True')
        
        for i in range(x):
            individual = np.array([np.random.randint(min_value_n, max_value_n) for i in range(0, n)])
            population.append(individual)

    return population

In [ ]:
def pop_to_dev(population):
  a = population
  a = np.array(a, dtype='float32', order='A')
  a_dev = cuda.to_device(a)
  return a_dev 

In [ ]:
@cuda.jit
def prod_sum_row(in_root, in_cos, out_sum, out_prod):
    sm1 = cuda.shared.array(threadsperblock, float32)
    sm2 = cuda.shared.array(threadsperblock, float32)
    bid = cuda.blockIdx.x
    tid = cuda.threadIdx.x
    bdim = cuda.blockDim.x
  #pamięc wspólna
    lid = tid
    sm1[lid] = 0
    sm2[lid] = 1
    while lid < in_root.shape[1]:
        sm1[tid] += in_root[bid, lid];
        sm2[tid] *= in_cos[bid, lid];
        lid += bdim
    cuda.syncthreads()

    sweep = bdim//2
    while sweep > 0:
        if tid < sweep:
            sm1[tid] += sm1[tid + sweep]
            sm2[tid] *= sm2[tid + sweep]
        sweep = sweep//2
        cuda.syncthreads()
    if tid == 0:
        out_sum[bid] = sm1[0]
        out_prod[bid] = sm2[0]

In [ ]:
threadsperblock = 256
rvecs1  = np.array([[1,2],[0,1]], dtype=np.float32)
rvecs2  = np.array([[1,2],[0,1]], dtype=np.float32)
d_rvecs1 = cuda.to_device(rvecs1)
d_rvecs2 = cuda.to_device(rvecs2)
d_sum = cuda.device_array(d_rvecs1.shape[0], dtype=np.float32)
d_prod = cuda.device_array(d_rvecs1.shape[0], dtype=np.float32)
prod_sum_row[d_rvecs1.shape[0], threadsperblock](d_rvecs1, d_rvecs2, d_sum, d_prod)



In [ ]:
@cuda.jit
def root_cos(x, out_root, out_cos):
  s1, s2 = cuda.grid(2)
  d1, d2 = cuda.gridsize(2)
  
  for i in range(s1, x.shape[0], d1):
    for j in range(s2, x.shape[1], d2):
      out_root[i,j] = x[i,j]**2
      out_cos[i,j] = math.cos(x[i,j]/(j+1))
  
  

In [ ]:
@cuda.jit
def func1ker(osobnik_sum, osobnik_prod, osobnik_value):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)
  for i in range(start, osobnik_sum.shape[0], stride):
    osobnik_value[i] = ((1/40)*osobnik_sum[i])+1-osobnik_prod[i]

In [ ]:
def func1(population):
  population = [i for i in population if i is not None]
  threadsperblock = 256
  #array do i z gpu
  pop_dev = pop_to_dev(population)
  out_root = cuda.device_array_like(pop_dev)
  out_cos = cuda.device_array_like(pop_dev)

  #oblicza root i cos dla genów, osobników w populacji
  root_cos[len(population),threadsperblock](pop_dev,out_root,out_cos)
  cuda.synchronize()

  #kolejne arraye 1d gpu, wyjście funckji liczącej prod i sum  
  out_sums = cuda.device_array(len(population), dtype = np.float32)
  out_prod = cuda.device_array(len(population), dtype = np.float32)

  prod_sum_row[len(population), threadsperblock](out_root, out_cos, out_sums, out_prod)

  cuda.synchronize()

  osobnik_value = cuda.device_array_like(out_sums)
  #wylicza wektor ocen
  func1ker[out_sums.shape[0],threadsperblock](out_sums, out_prod, osobnik_value)
  cuda.synchronize()
  scores = osobnik_value.copy_to_host()

  return scores 

In [ ]:
@cuda.jit
def sum_sum_row(in_root, in_cos, out_sum1, out_sum2):
    sm1 = cuda.shared.array(threadsperblock, float32)
    sm2 = cuda.shared.array(threadsperblock, float32)
    bid = cuda.blockIdx.x
    tid = cuda.threadIdx.x
    bdim = cuda.blockDim.x
  #pamięc wspólna
    lid = tid
    sm1[lid] = 0
    sm2[lid] = 0
    while lid < in_root.shape[1]:
        sm1[tid] += in_root[bid, lid];
        sm2[tid] += in_cos[bid, lid];
        lid += bdim
    cuda.syncthreads()

    sweep = bdim//2
    while sweep > 0:
        if tid < sweep:
            sm1[tid] += sm1[tid + sweep]
            sm2[tid] += sm2[tid + sweep]
        sweep = sweep//2
        cuda.syncthreads()
    if tid == 0:
        out_sum1[bid] = sm1[0]
        out_sum2[bid] = sm2[0]

In [ ]:
threadsperblock = 256
rvecs1  = np.array([[1,2],[0,1]], dtype=np.float32)
rvecs2  = np.array([[1,2],[0,1]], dtype=np.float32)
d_rvecs1 = cuda.to_device(rvecs1)
d_rvecs2 = cuda.to_device(rvecs2)
d_sum = cuda.device_array(d_rvecs1.shape[0], dtype=np.float32)
d_prod = cuda.device_array(d_rvecs1.shape[0], dtype=np.float32)
sum_sum_row[d_rvecs1.shape[0], threadsperblock](d_rvecs1, d_rvecs2, d_sum, d_prod)


In [ ]:
@cuda.jit
def func2ker(osobnik_sum1, osobnik_sum2, osobnik_value, n):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)
  for i in range(start, osobnik_sum1.shape[0], stride):
    osobnik_value[i] = 20*math.exp((-0.2)*(math.sqrt((1/n)*(osobnik_sum1[i]))))-math.exp((1/n)*osobnik_sum2[i])-20+math.e


In [ ]:
@cuda.jit
def root_cos2(x, out_root, out_cos):
  s1, s2 = cuda.grid(2)
  d1, d2 = cuda.gridsize(2)
  
  for i in range(s1, x.shape[0], d1):
    for j in range(s2, x.shape[1], d2):
      out_root[i,j] = x[i,j]**2
      out_cos[i,j] = math.cos(2*x[i,j]*math.pi)

In [ ]:
def func2(population):
  population = [i for i in population if i is not None]
  threadsperblock = 256
  #array do i z gpu
  pop_dev = pop_to_dev(population)
  n = pop_dev.shape[1]
  out_root = cuda.device_array_like(pop_dev)
  out_cos = cuda.device_array_like(pop_dev)

  #oblicza root i cos dla genów, osobników w populacji
  root_cos2[len(population),threadsperblock](pop_dev,out_root,out_cos)
  cuda.synchronize()

  #kolejne arraye 1d gpu, wyjście funckji liczącej prod i sum  
  out_sum1 = cuda.device_array(len(population), dtype = np.float32)
  out_sum2 = cuda.device_array(len(population), dtype = np.float32)

  sum_sum_row[len(population), threadsperblock](out_root, out_cos, out_sum1, out_sum2)

  cuda.synchronize()

  osobnik_value = cuda.device_array_like(out_sum1)

  #wylicza wektor ocen
  func2ker[out_sum1.shape[0],threadsperblock](out_sum1, out_sum2, osobnik_value, n)
  cuda.synchronize()
  scores = osobnik_value.copy_to_host()

  return scores 

In [ ]:
def crossing(indiv1, indiv2,n):
    '''
    indiv1

    indiv2
    '''
    #losowanie locus podzialu
    locus = np.random.randint(0, n)
    embryo1 = np.concatenate((indiv1[:locus],indiv2[locus:]))
    embryo2 = np.concatenate((indiv2[:locus],indiv1[locus:]))
    return embryo1, embryo2

In [ ]:
def mutate(individual, p_mutate):
    '''
    individuals : [int]
        lista genow
    p_mutate : float
        prawdopodobienstwo mutacji
    '''

    for i in range(0,len(individual)):
        r = np.random.rand()
        if r < p_mutate/2:
            individual[i] -= 1
        elif r < p_mutate:
            individual[i] += 1
    return individual

In [ ]:
def optimize(population, x, n, function, min_or_max,end_optim, p_mutate=0.2):
    '''
    population : list of numpy arrays
        zbiór wszystkich osobników
    '''
    best_individual_scores = []
    iteration = 0   
    while True:
        iteration += 1
        
        #krzyzowanie i mutacja
        for i in range(0, x, 2):
            crossing_result = crossing(population[i], population[i+1], n)
            for embryon in crossing_result:
                population.append(mutate(embryon,p_mutate))  

     
        scores = function(population)
        scores = scores.tolist()
  

        if min_or_max == 'max':
            reverse=True
        elif min_or_max == 'min':
            reverse=False
        else:
            raise Exception('min_or_max parameter invalid. Type "min" or "max".')
            
        population_score = sorted(zip(population, scores), key=lambda x:x[1], reverse=reverse)
        best_individual_gen = population_score[0][0]
        best_individual_score = population_score[0][1]
        
        best_individual_scores.append(best_individual_score)
            
        sorted_population, _ = zip(*population_score)
        population = list(sorted_population[:x])
        
        #print(f'Iteration: {iteration} | Best score: {best_individual_score}')
        
        # zakończ jeśli nie ma poprawy w 100 ostatnich iteracjach (?)
        if len(set(best_individual_scores[-end_optim:])) == 1 and iteration > 10:
            break
        
        
    return best_individual_gen, best_individual_score
        

In [ ]:
# Wielkość populacji
x = 800
# Liczba genów osobnika
n = 2
# Minimalna wartość genu
min_value_n = -30
# Maksymalna wartość genu
max_value_n = 30
# Inicjowanie osobnika zerami (w przypadku tych zadan nie ma sensu)
start_wth_zeros = False
# Pradopodobieństwo mutacji
p_mutate = 0.2
# Funkcja oceny
function = func2
# Cel (min/max)
min_or_max='max'
# Zakończ, jeśli wynik najlepszego osobnika nie zmienił się od ... iteracji.
end_optim=100

population = initial_pop_generation(x,n,start_wth_zeros,min_value_n,max_value_n)
best_gen, best_score = optimize(population,x,n,function, min_or_max,end_optim,p_mutate)
#print(f'Najlepszy osobnik: {best_gen}\n')
#print(f'Wynik dla najlepszego osobnika: {best_score}')

CZASY


In [ ]:
from time import perf_counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Liczba genów osobnika
n = 0
# Inicjowanie osobnika zerami (w przypadku tych zadan nie ma sensu)
start_wth_zeros = False
# Pradopodobieństwo mutacji
p_mutate = 0.2
# Zakończ, jeśli wynik najlepszego osobnika nie zmienił się od ... iteracji.
end_optim=100

In [ ]:

data = pd.DataFrame(columns=['Czas','Wynik','Odchylenie','GPU',
                             'Funkcja','Liczba genów','Wielkość populacji',
                             'Liczba procesów'])
iteracje = 10
ns = [2,10,20,50,100]
xs = [100,200,400,800,2000]

In [ ]:
def time_it(x, n, func, iteracje=10):
    best_scores = []
    times = []
    for _ in range(iteracje):
        t_start = perf_counter()
        population = initial_pop_generation(x,n,start_wth_zeros,min_value_n,max_value_n)
        best_gen, best_score = optimize(population,x,n,func, min_or_max,end_optim,p_mutate)
        t_stop = perf_counter()
        best_scores.append(round(best_score,2))
        times.append(t_stop-t_start)
    
    time = round(sum(times)/len(times),2)
    score = np.mean(best_scores)
    std_dev = np.std(best_scores)
    print('Sredni czas: ', time)
    print('Srednia ocena:', score)

    if func==func1:
        function = 1
    else:
        function = 2
    
    result = [time,score,std_dev,True,function,n,x,np.NaN]
    
    return result

In [ ]:
# Funkcja oceny
function = func1
# Cel (min/max)
min_or_max='min'
# Minimalna wartość genu
min_value_n = -40
# Maksymalna wartość genu
max_value_n = 40

In [ ]:
for x in xs:
  print('Populacja', x)
  for n in ns:
    print('Liczba genów: ', n)
    result = time_it(x,n,func1)
    data.loc[len(data)] = result

Populacja 100
Liczba genów:  2
Sredni czas:  0.25
Srednia ocena: 0.0
Liczba genów:  10
Sredni czas:  0.46
Srednia ocena: 0.0
Liczba genów:  20
Sredni czas:  0.86
Srednia ocena: 0.0
Liczba genów:  50
Sredni czas:  2.81
Srednia ocena: 0.213
Liczba genów:  100
Sredni czas:  5.67
Srednia ocena: 1.2129999999999999
Populacja 200
Liczba genów:  2
Sredni czas:  0.35
Srednia ocena: 0.0
Liczba genów:  10
Sredni czas:  0.55
Srednia ocena: 0.0
Liczba genów:  20
Sredni czas:  1.08
Srednia ocena: 0.0
Liczba genów:  50
Sredni czas:  3.96
Srednia ocena: 0.20400000000000001
Liczba genów:  100
Sredni czas:  9.83
Srednia ocena: 1.068
Populacja 400
Liczba genów:  2
Sredni czas:  0.37
Srednia ocena: 0.0
Liczba genów:  10
Sredni czas:  0.86
Srednia ocena: 0.0
Liczba genów:  20
Sredni czas:  1.76
Srednia ocena: 0.0
Liczba genów:  50
Sredni czas:  7.48
Srednia ocena: 0.18
Liczba genów:  100
Sredni czas:  15.92
Srednia ocena: 1.01
Populacja 800
Liczba genów:  2
Sredni czas:  0.73
Srednia ocena: 0.0
Liczba genó

In [ ]:
data

,Czas,Wynik,Odchylenie,GPU,Funkcja,Liczba genów,Wielkość populacji,Liczba procesów
0,0.25,0.000,0.000000,True,1,2,100,NaN
1,0.46,0.000,0.000000,True,1,10,100,NaN
2,0.86,0.000,0.000000,True,1,20,100,NaN
3,2.81,0.213,0.049204,True,1,50,100,NaN
4,5.67,1.213,0.293702,True,1,100,100,NaN
5,0.35,0.000,0.000000,True,1,2,200,NaN
6,0.55,0.000,0.000000,True,1,10,200,NaN
7,1.08,0.000,0.000000,True,1,20,200,NaN
8,3.96,0.204,0.033526,True,1,50,200,NaN
9,9.83,1.068,0.097550,True,1,100,200,NaN


In [ ]:
# Funkcja oceny
function = func2
# Cel (min/max)
min_or_max='max'
# Minimalna wartość genu
min_value_n = -30
# Maksymalna wartość genu
max_value_n = 30

In [ ]:
for x in xs:
  print('Populacja', x)
  for n in ns:
    print('Liczba genów: ', n)
    result = time_it(x,n,func2)
    data.loc[len(data)] = result

Populacja 100
Liczba genów:  2
Sredni czas:  0.25
Srednia ocena: 0.0
Liczba genów:  10
Sredni czas:  0.4
Srednia ocena: 0.0
Liczba genów:  20
Sredni czas:  0.74
Srednia ocena: 0.0
Liczba genów:  50
Sredni czas:  2.08
Srednia ocena: -1.568
Liczba genów:  100
Sredni czas:  4.77
Srednia ocena: -2.395
Populacja 200
Liczba genów:  2
Sredni czas:  0.33
Srednia ocena: 0.0
Liczba genów:  10
Sredni czas:  0.53
Srednia ocena: 0.0
Liczba genów:  20
Sredni czas:  0.97
Srednia ocena: 0.0
Liczba genów:  50
Sredni czas:  3.17
Srednia ocena: -1.4769999999999999
Liczba genów:  100
Sredni czas:  7.39
Srednia ocena: -2.3489999999999998
Populacja 400
Liczba genów:  2
Sredni czas:  0.45
Srednia ocena: 0.0
Liczba genów:  10
Sredni czas:  0.84
Srednia ocena: 0.0
Liczba genów:  20
Sredni czas:  1.46
Srednia ocena: 0.0
Liczba genów:  50
Sredni czas:  5.07
Srednia ocena: -1.454
Liczba genów:  100
Sredni czas:  14.16
Srednia ocena: -2.275
Populacja 800
Liczba genów:  2
Sredni czas:  0.64
Srednia ocena: 0.0
Liczb

In [ ]:
data

,Czas,Wynik,Odchylenie,GPU,Funkcja,Liczba genów,Wielkość populacji,Liczba procesów
0,0.25,0.000,0.000000,True,1,2,100,NaN
1,0.46,0.000,0.000000,True,1,10,100,NaN
2,0.86,0.000,0.000000,True,1,20,100,NaN
3,2.81,0.213,0.049204,True,1,50,100,NaN
4,5.67,1.213,0.293702,True,1,100,100,NaN
5,0.35,0.000,0.000000,True,1,2,200,NaN
6,0.55,0.000,0.000000,True,1,10,200,NaN
7,1.08,0.000,0.000000,True,1,20,200,NaN
8,3.96,0.204,0.033526,True,1,50,200,NaN
9,9.83,1.068,0.097550,True,1,100,200,NaN
